# Memory usage measurements

In [1]:
import sys
import subprocess
import shutil
import tempfile
from pathlib import Path

assert shutil.which('valgrind')
assert shutil.which('ms_print')

CWD = Path.cwd()

def invoke_massif(args, output):
  subprocess.run([
    'valgrind', '--tool=massif', f'--massif-out-file={output}'
  ]  + args, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=CWD)


def memory_graph(test_name, code):
  massif_output = test_name + '.massif'
  cwd = Path().cwd()
  invoke_massif([
    sys.executable, '-c', ';'.join(code.split('\n'))
  ], massif_output)
  return '\n'.join(subprocess.check_output(
    ['ms_print', massif_output], cwd=CWD
  ).decode().split('\n\n')[:2])


In [2]:
random_matr = '''\
import numpy as np
n = 4096
m = 2 * n
matr = np.zeros((n, m), dtype=np.complex128)
matr = np.random.rand(n, m)
'''

Numpy:

In [3]:
numpy_test = random_matr + '''\
import numpy
expected = numpy.fft.fft2(matr)
print(expected)
'''

print(memory_graph('numpy', numpy_test))

--------------------------------------------------------------------------------
Command:            /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import numpy as np;n = 4096;m = 2 * n;matr = np.zeros((n, m), dtype=np.complex128);matr = np.random.rand(n, m);import numpy;expected = numpy.fft.fft2(matr);print(expected);
Massif arguments:   --massif-out-file=numpy.massif
ms_print arguments: numpy.massif
--------------------------------------------------------------------------------

    GB
1.254^                                                    #                   
     |                                                    #:::::::::::::::::::
     |                                                    #::::::::::::::: :::
     |                                                    #::::::::::::::: :::
     |                                                    #::::::::::::::: :::
     |                                                    #::::::::::::::: :::
     |              

perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LC_TERMINAL = "iTerm2",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


FFT with type conversion:

In [4]:
fft_type_conversion = random_matr + '''\
import fft_project
expected = fft_project.fft2d(matr, return_copy=True, use_threads=False)
print(expected)
'''

print(memory_graph('type_conversions', fft_type_conversion))

--------------------------------------------------------------------------------
Command:            /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import numpy as np;n = 4096;m = 2 * n;matr = np.zeros((n, m), dtype=np.complex128);matr = np.random.rand(n, m);import fft_project;expected = fft_project.fft2d(matr, return_copy=True, use_threads=False);print(expected);
Massif arguments:   --massif-out-file=type_conversions.massif
ms_print arguments: type_conversions.massif
--------------------------------------------------------------------------------

    MB
772.0^                     :::::::::::::::::::::::::::::::::::::::::::::::::: 
     |                  #:::                                                 :
     |                  #: :                                                 :
     |                  #: :                                                 :
     |                  #: :                                                 :
     |                  #: :   

perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LC_TERMINAL = "iTerm2",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").


FFT with no type conversions

In [5]:
fft_no_type_conv = random_matr + '''\
import fft_project
expected = fft_project.fft2d(matr, return_copy=False, use_threads=False)
print(expected)
'''

print(memory_graph('no_type_conv', fft_no_type_conv))

--------------------------------------------------------------------------------
Command:            /home/gyzavyalov-rpi/tree/public/2d-fft/venv/bin/python -c import numpy as np;n = 4096;m = 2 * n;matr = np.zeros((n, m), dtype=np.complex128);matr = np.random.rand(n, m);import fft_project;expected = fft_project.fft2d(matr, return_copy=False, use_threads=False);print(expected);
Massif arguments:   --massif-out-file=no_type_conv.massif
ms_print arguments: no_type_conv.massif
--------------------------------------------------------------------------------

    MB
771.8^                                                                       #
     |                                                                       #
     |                                                                       #
     |                                                                       #
     |                                                                       #
     |                                

perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = "en_US.UTF-8",
	LC_CTYPE = "en_US.UTF-8",
	LC_TERMINAL = "iTerm2",
	LANG = "en_GB.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_GB.UTF-8").
